# Overview

This notebook was used to prepare the Uniprot `features_file.tsv` to use for Cluster Mode of the ProteinCartography pipeline.

In order to use this notebook, you must clone the [ProteinCartography GitHub repository](https://github.com/Arcadia-Science/ProteinCartography) and activate the `cartography` conda env. However, you should be within the `2023-actin-embedding/notebooks` folder.

The original list of actins was generated in our [Defining Actin](https://research.arcadiascience.com/pub/idea-defining-actin/release/4?readingCollection=9a516d32) pub and the corresponding [Actin Prediction](https://github.com/Arcadia-Science/2022-actin-prediction) pipeline, where we did a protein BLAST search for 50,000 matches to [human beta-actin](https://www.uniprot.org/uniprotkb/P60709/entry). The list can be found in this [Zenodo archive](https://zenodo.org/records/7384393).


## Setup

Import dependencies.


In [1]:
import os
import pandas as pd
import sys

PC_path = "./../../ProteinCartography/ProteinCartography"
input_path = "./../input"
sys.path.append(PC_path)

Prepare directories. Once these are prepared, place the 2022-actin-prediction-blasoutputs.txt files in the `~/Actin/prep` folder.


In [2]:
os.makedirs("./../output/", exist_ok=True)

## Map Refseq IDs

The output of BLAST is a list of Refseq IDs, but ProteinCartography and the AlphaFold database reference proteins based on their Accessions/UniProt IDs. This step converts Refseq IDs to Accessions/UniProt IDs and then uses them to download the required metadata file. Note that I split this into 2 batches as the single batch was too large and continually failed.


In [4]:
# Set paths

map_refseqids = os.path.join(PC_path, "map_refseqids.py")
input_map_refseqids1 = os.path.join(
    input_path, "2022-actin-prediction-blastoutputs1.txt"
)
input_map_refseqids2 = os.path.join(
    input_path, "2022-actin-prediction-blastoutputs1.txt"
)
output_map_refseqids1 = os.path.join(input_path, "blastoutput_uniprot1.txt")
output_map_refseqids2 = os.path.join(input_path, "blastoutput_uniprot2.txt")

# Batch 1
os.system(
    f"python {map_refseqids} -i {input_map_refseqids1} -o {output_map_refseqids1}"
)

# Batch 2
os.system(
    f"python {map_refseqids} -i {input_map_refseqids2} -o {output_map_refseqids2}"
)

0

## Download UniProt metadata

After obtaining UniProt IDs, we use them to download metadata from UniProt. This data will be the bulk of the UniProt features file.


In [5]:
# Set paths

fetch_uniprot_metadata = os.path.join(PC_path, "fetch_uniprot_metadata.py")
output_uniprot_metadata1 = os.path.join(input_path, "uniprot_features1.tsv")
output_uniprot_metadata2 = os.path.join(input_path, "uniprot_features2.tsv")

# Batch 1
os.system(
    f"python {fetch_uniprot_metadata} -i {output_map_refseqids1} -o {output_uniprot_metadata1}"
)

# Batch 2
os.system(
    f"python {fetch_uniprot_metadata} -i {output_map_refseqids2} -o {output_uniprot_metadata2}"
)


>> Starting batch 1 of 56
downloaded 300 / 16614 hits for batch 1
>> Starting batch 2 of 56
downloaded 300 / 16614 hits for batch 2
>> Starting batch 3 of 56
downloaded 300 / 16614 hits for batch 3
>> Starting batch 4 of 56
downloaded 300 / 16614 hits for batch 4
>> Starting batch 5 of 56
downloaded 300 / 16614 hits for batch 5
>> Starting batch 6 of 56
downloaded 300 / 16614 hits for batch 6
>> Starting batch 7 of 56
downloaded 300 / 16614 hits for batch 7
>> Starting batch 8 of 56
downloaded 300 / 16614 hits for batch 8
>> Starting batch 9 of 56
downloaded 300 / 16614 hits for batch 9
>> Starting batch 10 of 56
downloaded 300 / 16614 hits for batch 10
>> Starting batch 11 of 56
downloaded 300 / 16614 hits for batch 11
>> Starting batch 12 of 56
downloaded 300 / 16614 hits for batch 12
>> Starting batch 13 of 56
downloaded 300 / 16614 hits for batch 13
>> Starting batch 14 of 56
downloaded 300 / 16614 hits for batch 14
>> Starting batch 15 of 56
downloaded 300 / 16614 hits for batch 1

0

In [25]:
# Merge batched files
uniprot_features1 = pd.read_csv("../input/uniprot_features1.tsv", sep="\t")
uniprot_features2 = pd.read_csv("../input/uniprot_features2.tsv", sep="\t")
uniprot_features_combined = pd.concat([uniprot_features1, uniprot_features2])
uniprot_features_combined = uniprot_features_combined.drop_duplicates(subset="protid", keep="first")

# Save uniprot_features file
uniprot_features_combined.to_csv(
    "../input/uniprot_features_combined.tsv", sep="\t", index=None
)

## Filter UniProt hits

We filtered UniProt hits based on fragment status and whether or not the UniProt entry was active using the `filter_uniprot_hits.py` script from [ProteinCartography](https://github.com/Arcadia-Science/ProteinCartography).


In [26]:
# Set paths
filter_uniprot_hits = os.path.join(PC_path, "filter_uniprot_hits.py")
output_filtered_list = os.path.join(input_path, "features_filter.tsv")
input_uniprot_features = os.path.join(input_path, "uniprot_features_combined.tsv")

os.system(
    f"python {filter_uniprot_hits} -i {input_uniprot_features} -o {output_filtered_list}"
)

0

In [27]:
# Renames first column to protid
features_filtered = pd.read_csv(output_filtered_list, sep="\t", names=["protid"])

# Save uniprot_features file
features_filtered.to_csv(output_filtered_list, sep="\t", index=None)
display(features_filtered)

,protid
0,A5A3E0
1,A0A816JW31
2,A0A0V1PKQ0
3,A0A4D9DGK8
4,A0A068XZE9
...,...
11092,A0A921QVQ9
11093,A0A9J7N7R5
11094,A9P810
11095,D8LXR4


In [28]:
# Prep paths
output_filter_hits = os.path.join(input_path, "uniprot_features_filtered.tsv")

# Apply filter
uniprot_features_combined = pd.read_csv("../input/uniprot_features_combined.tsv", sep="\t")
uniprot_features_filtered = uniprot_features_combined.merge(
    features_filtered, on="protid", how="inner"
)

# Save uniprot_features file
uniprot_features_filtered.to_csv(output_filter_hits, sep="\t", index=None)

## Reformat features file

The features file from UniProt must be reformatted slightly to work well with ProteinCartography. We reformatted the file to fit with guidelines listed [here](https://github.com/Arcadia-Science/ProteinCartography#feature-file-main-columns).


In [29]:
# Read in raw features file

uniprot_features_filtered = pd.read_csv(output_filter_hits, sep="\t")

# Reformat lineage column

lineage_string_splitter = lambda lineage_string: [
    rank.split(" (")[0] for rank in lineage_string.split(", ")
]

uniprot_features_filtered["Lineage"] = uniprot_features_filtered[
    "Taxonomic lineage"
].apply(lineage_string_splitter)

# Saves updated uniprot_features file

uniprot_features_filtered.to_csv("../output/uniprot_features.tsv", sep="\t", index=None)